## Introduction to LangChain

In [2]:
# For Colab user
import os
from google.colab import userdata
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

In [3]:
# download sample data
!git clone https://github.com/manote101/Building-Apps-with-NIM.git

fatal: destination path 'Building-Apps-with-NIM' already exists and is not an empty directory.


In [ ]:
!pip install -r Building-Apps-with-NIM/requirements.txt

In [4]:
pip install -q --upgrade langchain langchain-community langchain-nvidia_ai_endpoints faiss-cpu requests==2.32.4

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnableMap, RunnablePassthrough

# 1. Load and split documents
loader = TextLoader("Building-Apps-with-NIM/data/doc1.txt")  # replace with your filecd
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 2. Create embeddings and vectorstore
embeddings = NVIDIAEmbeddings(model="nvidia/llama-3.2-nv-embedqa-1b-v2")
vectorstore = FAISS.from_documents(splits, embeddings)

# 3. Create retriever
retriever = vectorstore.as_retriever()

# 4. Create LLM and prompt
llm = ChatNVIDIA(model="meta/llama-3.2-3b-instruct", temperature=0)

prompt = ChatPromptTemplate.from_template("""
You are a helpful AI assistant. Use the following context to answer the user's question.

Context:
{context}

Question:
{question}

Answer:
""")

# 5. Create a retrieval-augmented chain
retrieval_chain = RunnableMap({
    # "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "context": lambda x: retriever.invoke(x["question"]),  # Use invoke()
    "question": RunnablePassthrough(),  # Pass the question unchanged
}) | prompt | llm

# 6. Call invoke() to run it
result = retrieval_chain.invoke({"question": "บริษัทใดมีการนำ Nemo microservices มาใข้แล้วบ้าง"})

# 7. Print result
print(result.content)


ตามข้อมูลที่ให้มา บริษัทต่อไปนี้มีการนำ NeMo microservices มาใช้แล้ว:

1. Cloudera
2. Datadog
3. Dataiku
4. DataRobot
5. DataStax
6. SuperAnnotate AI
7. Weights & Biases
8. AT&T (ใช้ NeMo Customizer และ Evaluator เพื่อเพิ่มความแม่นยำของ AI Agent โดย fine-tune โมเดล Mistral 7B สำหรับบริการส่วนบุคคล)
